In [ ]:
!pip install transformers


In [3]:
import zipfile
import os

# Unzip uploaded file
with zipfile.ZipFile("iob_output.zip", 'r') as zip_ref:
    zip_ref.extractall("iob_data")

# Check file names
file_names = os.listdir("iob_data")
print("Number of files:", len(file_names))
print("Example file:", file_names[0])


Number of files: 97
Example file: 16579849.txt


In [4]:
import json

def load_iob_data_from_folder(folder_path):
    all_sentences = []
    for fname in os.listdir(folder_path):
        if fname.endswith(".txt"):
            with open(os.path.join(folder_path, fname), "r", encoding="utf-8") as f:
                data = json.load(f)
                all_sentences.extend(data)  # each file has many sentences
    return all_sentences

sentences = load_iob_data_from_folder("iob_data")
print("Total sentences loaded:", len(sentences))
print("Example:", sentences[0][:5])


Total sentences loaded: 28559
Example: [['Production', 'O'], ['and', 'O'], ['characterization', 'O'], ['of', 'O'], ['murine', 'O']]


In [5]:
from sklearn.model_selection import train_test_split

# Split sentences into train and test (80% train, 20% test)
train_sentences, test_sentences = train_test_split(sentences, test_size=0.2, random_state=42)

# Further split train_sentences into train and validation (80% train, 20% validation)
train_sentences, val_sentences = train_test_split(train_sentences, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_sentences)}")
print(f"Validation set size: {len(val_sentences)}")
print(f"Test set size: {len(test_sentences)}")


Training set size: 18277
Validation set size: 4570
Test set size: 5712


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
def tokenize_and_chunk(sentences, tokenizer, max_len=512, stride=256, pad_label="PAD"):
    tokenized_inputs = []
    aligned_labels = []

    for sentence in sentences:
        words, labels = zip(*sentence)
        tokens = []
        token_labels = []

        for word, label in zip(words, labels):
            word_pieces = tokenizer.tokenize(word)
            tokens.extend(word_pieces)
            token_labels.extend([label] + [pad_label] * (len(word_pieces) - 1))

        # Chunking logic
        start = 0
        while start < len(tokens):
            end = min(start + max_len, len(tokens))
            chunk_tokens = tokens[start:end]
            chunk_labels = token_labels[start:end]

            # Convert to IDs
            input_ids = tokenizer.convert_tokens_to_ids(chunk_tokens)

            tokenized_inputs.append(input_ids)
            aligned_labels.append(chunk_labels)

            if end == len(tokens):
                break
            start += stride

    return tokenized_inputs, aligned_labels

# Now, tokenize and chunk for each of the splits
train_token_ids, train_label_ids = tokenize_and_chunk(train_sentences, tokenizer)
val_token_ids, val_label_ids = tokenize_and_chunk(val_sentences, tokenizer)
test_token_ids, test_label_ids = tokenize_and_chunk(test_sentences, tokenizer)

# Optional: Check the first example from each set to confirm
print("Example train token IDs:", train_token_ids[0][:10])
print("Corresponding train labels:", train_label_ids[0][:10])

print("Example validation token IDs:", val_token_ids[0][:10])
print("Corresponding validation labels:", val_label_ids[0][:10])

print("Example test token IDs:", test_token_ids[0][:10])
print("Corresponding test labels:", test_label_ids[0][:10])



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Example train token IDs: [1143, 1306, 1884, 118, 10187, 1104, 1103, 2025, 1105, 6842]
Corresponding train labels: ['O', 'PAD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Example validation token IDs: [174, 1477, 2087, 1475, 1108, 1145, 11168, 1107, 1241, 4272]
Corresponding validation labels: ['O', 'PAD', 'PAD', 'PAD', 'O', 'O', 'O', 'O', 'O', 'B']
Example test token IDs: [9279, 1167, 27021, 1108, 1562, 1107, 1103, 9455, 7301, 9084]
Corresponding test labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PAD', 'PAD']


In [ ]:
!pip install torch


In [7]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Function to convert inputs and labels to tensors and pad them
def prepare_data(token_ids, label_ids, tokenizer):
    # 1. Convert input and label sequences to tensors
    input_id_tensors = [torch.tensor(seq) for seq in token_ids]
    label_map = {"O": 0, "B": 1, "I": 2}

    label_id_tensors = [
        torch.tensor([
            label_map[lbl] if lbl in label_map else -100  # <- safely handles 'PAD' or others
            for lbl in labels
        ])
        for labels in label_ids
    ]

    # 2. Pad sequences
    padded_inputs = pad_sequence(input_id_tensors, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_labels = pad_sequence(label_id_tensors, batch_first=True, padding_value=-100)  # -100 ignored in loss

    # 3. Create attention masks
    attention_masks = (padded_inputs != tokenizer.pad_token_id).long()

    return padded_inputs, padded_labels, attention_masks

# Prepare the data for each split
train_padded_inputs, train_padded_labels, train_attention_masks = prepare_data(train_token_ids, train_label_ids, tokenizer)
val_padded_inputs, val_padded_labels, val_attention_masks = prepare_data(val_token_ids, val_label_ids, tokenizer)
test_padded_inputs, test_padded_labels, test_attention_masks = prepare_data(test_token_ids, test_label_ids, tokenizer)

# Check shapes for each split
print("Training set shapes:")
print("Padded inputs shape:", train_padded_inputs.shape)
print("Padded labels shape:", train_padded_labels.shape)
print("Attention masks shape:", train_attention_masks.shape)

print("\nValidation set shapes:")
print("Padded inputs shape:", val_padded_inputs.shape)
print("Padded labels shape:", val_padded_labels.shape)
print("Attention masks shape:", val_attention_masks.shape)

print("\nTest set shapes:")
print("Padded inputs shape:", test_padded_inputs.shape)
print("Padded labels shape:", test_padded_labels.shape)
print("Attention masks shape:", test_attention_masks.shape)


Training set shapes:
Padded inputs shape: torch.Size([18280, 512])
Padded labels shape: torch.Size([18280, 512])
Attention masks shape: torch.Size([18280, 512])

Validation set shapes:
Padded inputs shape: torch.Size([4570, 303])
Padded labels shape: torch.Size([4570, 303])
Attention masks shape: torch.Size([4570, 303])

Test set shapes:
Padded inputs shape: torch.Size([5712, 351])
Padded labels shape: torch.Size([5712, 351])
Attention masks shape: torch.Size([5712, 351])


In [8]:
!pip install tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=bd0af6e03cf60adb15dd792714e2e6ff48c1833aa647845b1bf30ab959539c66
  Stored in directory: /root/.cache/pip/wheels/c8/c7/30/e5935be2cfa6883be72462333edc414d8928f3c78eaabec38a
Successfully built tdqm


In [24]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel
from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm
import itertools
EPOCHS = 3

# Dataset class
class NERDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }
# Validation function
def evaluate(model, dataloader, device, criterion):
    model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="  Validating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits.view(-1, 3), labels.view(-1))
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1).view(-1)
            true  = labels.view(-1)
            mask  = (true != -100) & (true != 0)

            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(true[mask].cpu().numpy())

    # compute B/I macro metrics
    filtered_preds  = [p for p,t in zip(all_preds, all_labels) if t in [1,2]]
    filtered_labels = [t for t in all_labels   if t in [1,2]]
    precision, recall, f1, _ = precision_recall_fscore_support(
        filtered_labels, filtered_preds, labels=[1,2], average="macro"
    )
    avg_loss = total_loss / len(dataloader)
    return avg_loss, precision, recall, f1

# Model class
class BioBERT_BiGRU_NER(nn.Module):
    def __init__(self, bert_model_name, hidden_dim, num_labels=3, dropout_rate=0.3):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.gru  = nn.GRU(input_size=self.bert.config.hidden_size,
                           hidden_size=hidden_dim,
                           batch_first=True,
                           bidirectional=True)
        self.dropout    = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(hidden_dim * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state
        gru_output, _ = self.gru(embeddings)
        dropped = self.dropout(gru_output)
        return self.classifier(dropped)


# Hyperparameter grid
param_grid = {
    "learning_rate": [5e-5, 2e-5],
    "hidden_dim":    [64, 128],
    "dropout":       [0.1, 0.3],
}

# generate all combinations
all_configs = list(itertools.product(
    param_grid["learning_rate"],
    param_grid["hidden_dim"],
    param_grid["dropout"],
))
# Create dataset objects
train_dataset = NERDataset(train_padded_inputs, train_attention_masks, train_padded_labels)
val_dataset   = NERDataset(val_padded_inputs, val_attention_masks, val_padded_labels)
test_dataset  = NERDataset(test_padded_inputs, test_attention_masks, test_padded_labels)

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader   = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_dataloader  = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [ ]:

best_f1 = 0.0
best_config = None

# Grid search for training all configurations
for lr, hid_dim, do_rate in all_configs:
    print(f"\n>>> Config: lr={lr}, hidden_dim={hid_dim}, dropout={do_rate}")

    # init model, criterion, optimizer
    model = BioBERT_BiGRU_NER("dmis-lab/biobert-base-cased-v1.1",
                              hidden_dim=hid_dim,
                              dropout_rate=do_rate).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    # train + validate
    for epoch in range(1, EPOCHS+1):
        model.train()
        train_loss = 0.0

        # training pass
        progress = tqdm(train_dataloader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False)
        for batch in progress:
            input_ids = batch['input_ids'].to(device)
            mask      = batch['attention_mask'].to(device)
            labels    = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, mask)
            loss   = criterion(logits.view(-1, 3), labels.view(-1))
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            progress.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / len(train_dataloader)
        val_loss, precision, recall, f1 = evaluate(model, val_dataloader, device, criterion)

        print(f"  Epoch {epoch} → Train Loss: {avg_train_loss:.4f}  │  Val Loss: {val_loss:.4f}  │  F1(B/I): {f1:.4f}")

    # if this config is best so far, save model
    if f1 > best_f1:
        best_f1 = f1
        best_config = (lr, hid_dim, do_rate)
        torch.save(model.state_dict(), "best_model.pt")
        print(f"  >>> New best! Saved with F1={f1:.4f}")

print(f"\n=== Best config: lr={best_config[0]}, hidden_dim={best_config[1]}, dropout={best_config[2]} with F1={best_f1:.4f} ===")


Device: cuda

>>> Config: lr=5e-05, hidden_dim=64, dropout=0.1


  Epoch 1 → Train Loss: 0.1490  │  Val Loss: 0.1002  │  F1(B/I): 0.0822


  Epoch 2 → Train Loss: 0.0966  │  Val Loss: 0.0800  │  F1(B/I): 0.3619


  Epoch 3 → Train Loss: 0.0840  │  Val Loss: 0.0715  │  F1(B/I): 0.4667
  >>> New best! Saved with F1=0.4667

>>> Config: lr=5e-05, hidden_dim=64, dropout=0.3


  Epoch 1 → Train Loss: 0.1581  │  Val Loss: 0.1024  │  F1(B/I): 0.0846


...

  Epoch 2 → Train Loss: 0.0895  │  Val Loss: 0.0759  │  F1(B/I): 0.5392


  Epoch 3 → Train Loss: 0.0783  │  Val Loss: 0.0663  │  F1(B/I): 0.5393

>>> Config: lr=2e-05, hidden_dim=64, dropout=0.1


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  Epoch 1 → Train Loss: 0.2198  │  Val Loss: 0.1356  │  F1(B/I): 0.0000


Epoch 2/3:  55%|█████▌    | 1266/2285 [05:06<04:08,  4.11it/s, loss=0.0937]

Load Saved

In [21]:
!ls -lh "best_model_config_5_128_.pt"


-rw-r--r-- 1 root root 382M Apr 22 01:44 best_model_config_5_128_.pt


In [ ]:
# manually set best_config
lr = 5e-5
hid_dim = 128
do_rate = 0.1
best_config = (lr, hid_dim, do_rate)
model = BioBERT_BiGRU_NER(
    bert_model_name="dmis-lab/biobert-base-cased-v1.1",
    hidden_dim=best_config[1],
    dropout_rate=best_config[2]
).to(device)

model.load_state_dict(torch.load("biobert_biGRU_ner_model_trained.pth", weights_only=False, map_location=torch.device('cpu')))
model.eval()

# Extract and save embeddings for a chosen split
from torch.utils.data import DataLoader

# Choose split: train_dataset, val_dataset or test_dataset
dataset = val_dataset
loader  = DataLoader(dataset, batch_size=8, shuffle=False)

all_embs, all_labels = [], []

with torch.no_grad():
    for batch in tqdm(loader, desc="Extracting embeddings"):
        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"]

        # Only get the BioBERT embeddings (before GRU/classifier)
        bert_out   = model.bert(input_ids=input_ids, attention_mask=attention_mask)
        embs       = bert_out.last_hidden_state.cpu()  # (B, seq_len, hidden_size)

        all_embs.append(embs)
        all_labels.append(labels)

# Concatenate and save
emb_tensor = torch.cat(all_embs,  dim=0)  # (N, seq_len, hidden_size)
lbl_tensor = torch.cat(all_labels, dim=0)  # (N, seq_len)

torch.save(emb_tensor, "val_embeddings.pt")
torch.save(lbl_tensor, "val_labels.pt")
print("Saved:", emb_tensor.shape, lbl_tensor.shape)

from google.colab import files
files.download("val_embeddings.pt")
files.download("val_labels.pt")
files.download("best_model.pt")


Extracting embeddings:   1%|          | 5/572 [00:52<1:40:40, 10.65s/it]

# IGNORE ALL CELLS BELOW THIS

In [ ]:
# === Extract embeddings from BioBERT ===
def extract_embeddings_and_labels(dataloader):
    all_embeddings = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state.cpu()
            all_embeddings.extend(embeddings)
            all_labels.extend(labels)

    return all_embeddings, all_labels

train_embeddings, train_labels = extract_embeddings_and_labels(train_dataloader)
val_embeddings, val_labels = extract_embeddings_and_labels(val_dataloader)
test_embeddings, test_labels = extract_embeddings_and_labels(test_dataloader)

# === Define tag2idx and idx2tag ===
tag2idx = {"O": 0, "B": 1, "I": 2}
idx2tag = {v: k for k, v in tag2idx.items()}

# === Save embeddings and labels ===
import os, json
os.makedirs("embeddings", exist_ok=True)
torch.save(train_embeddings, "embeddings/train_embeddings.pt")
torch.save(val_embeddings, "embeddings/val_embeddings.pt")
torch.save(test_embeddings, "embeddings/test_embeddings.pt")

torch.save(train_labels, "embeddings/train_labels.pt")
torch.save(val_labels, "embeddings/val_labels.pt")
torch.save(test_labels, "embeddings/test_labels.pt")

with open("embeddings/tag2idx.json", "w") as f:
    json.dump(tag2idx, f)
with open("embeddings/idx2tag.json", "w") as f:
    json.dump(idx2tag, f)

print("✅ Embeddings and labels saved.")



In [ ]:
from google.colab import files
files.download('/content/best_model.pt')

In [ ]:
# Save the entire model (architecture + trained weights)
torch.save(model.state_dict(), 'biobert_biGRU_ner_model_trained_1.pth')
# # Save model and optimizer state (for resuming training)
# torch.save({
#     'epoch': epoch,
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'loss': loss.item(),
# }, 'checkpoint.pth')


In [ ]:
from google.colab import files
files.download('/content/biobert_biGRU_ner_model_trained.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>